In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-12-01 15:18:54.014358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 15:18:54.116463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 15:18:54.116478: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-01 15:18:54.137196: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-01 15:18:54.668273: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-12-01 15:18:58.862234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-01 15:18:58.862907: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-12-01 15:18:58.862972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-12-01 15:18:58.863028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-12-01 15:18:58.863082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 387ms/step - loss: 2.7384 - auc: 0.2897
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6681 - auc: 0.6938
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5167 - auc: 0.8462
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6637 - auc: 0.8597
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7293 - auc: 0.8745
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7371 - auc: 0.8753
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6988 - auc: 0.8791
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5721 - auc: 0.8924
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4212 - auc: 0.9032
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2840 - auc: 0.9091
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1412 

1/1 [==============================] - 0s 28ms/step - loss: 0.2453 - auc: 0.9914
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2524 - auc: 0.9899
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2525 - auc: 0.9895
Epoch 91/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2460 - auc: 0.9903
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2459 - auc: 0.9901
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2461 - auc: 0.9903
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2445 - auc: 0.9909
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2380 - auc: 0.9909
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2449 - auc: 0.9900
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2409 - auc: 0.9906
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 1.6575 - auc: 0.6173
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5246 - auc: 0.8045
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5155 - auc: 0.8238
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4012 - auc: 0.8331
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2857 - auc: 0.8370
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1986 - auc: 0.8320
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1522 - auc: 0.8251
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1272 - auc: 0.8211
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.0426 - auc: 0.8591
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 0.9405 - auc: 0.9038
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2733 - auc: 0.9865
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2761 - auc: 0.9861
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2581 - auc: 0.9903
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2702 - auc: 0.9870
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2730 - auc: 0.9860
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2751 - auc: 0.9861
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2723 - auc: 0.9866
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2698 - auc: 0.9870
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2671 - auc: 0.9874
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2603 - auc: 0.9890
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.5965 - auc: 0.7610
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4766 - auc: 0.7285
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3462 - auc: 0.8155
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2814 - auc: 0.8465
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1834 - auc: 0.8667
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0744 - auc: 0.8901
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0170 - auc: 0.9015
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 0.9488 - auc: 0.9207
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 0.8827 - auc: 0.9400
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.8361 - auc: 0.9505
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2289 - auc: 0.9916
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2250 - auc: 0.9917
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2261 - auc: 0.9919
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2265 - auc: 0.9918
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2281 - auc: 0.9911
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2319 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2264 - auc: 0.9920
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2211 - auc: 0.9925
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2133 - auc: 0.9937
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2186 - auc: 0.9928
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [6]:
m='remove_Kappa'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 339ms/step - loss: 1.5631 - auc: 0.7402
Epoch 2/100
1/1 [==============================] - 0s 23ms/step - loss: 1.4633 - auc: 0.8154
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3053 - auc: 0.8327
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2094 - auc: 0.8414
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1443 - auc: 0.8605
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0280 - auc: 0.9088
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9742 - auc: 0.9272
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9213 - auc: 0.9399
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8572 - auc: 0.9523
Epoch 10/100
1/1 [==============================] - 0s 25ms/step - loss: 0.8151 - auc: 0.9578
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2280 - auc: 0.9919
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2282 - auc: 0.9916
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2328 - auc: 0.9911
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2300 - auc: 0.9911
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2314 - auc: 0.9910
Epoch 93/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2219 - auc: 0.9922
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2297 - auc: 0.9910
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2295 - auc: 0.9918
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2220 - auc: 0.9929
Epoch 97/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2265 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 392ms/step - loss: 1.6535 - auc: 0.6675
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6581 - auc: 0.7945
Epoch 3/100
1/1 [==============================] - 0s 22ms/step - loss: 1.5393 - auc: 0.8073
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3759 - auc: 0.8142
Epoch 5/100
1/1 [==============================] - 0s 41ms/step - loss: 1.2966 - auc: 0.7929
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 1.2259 - auc: 0.7984
Epoch 7/100
1/1 [==============================] - 0s 44ms/step - loss: 1.1447 - auc: 0.8288
Epoch 8/100
1/1 [==============================] - 0s 43ms/step - loss: 1.0838 - auc: 0.8590
Epoch 9/100
1/1 [==============================] - 0s 40ms/step - loss: 1.0008 - auc: 0.8962
Epoch 10/100
1/1 [==============================] - 0s 46ms/step - loss: 0.9655 - auc: 0.9100
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2818 - auc: 0.9848
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2929 - auc: 0.9816
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2694 - auc: 0.9869
Epoch 91/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2705 - auc: 0.9871
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2876 - auc: 0.9829
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2754 - auc: 0.9854
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2766 - auc: 0.9854
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2791 - auc: 0.9842
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2663 - auc: 0.9872
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2773 - auc: 0.9854
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.8677 - auc: 0.2418
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6595 - auc: 0.6106
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4161 - auc: 0.8191
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5168 - auc: 0.8496
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6225 - auc: 0.8572
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6943 - auc: 0.8554
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6677 - auc: 0.8628
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5476 - auc: 0.8765
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4594 - auc: 0.8818
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3283 - auc: 0.8838
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2788 - auc: 0.9871
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2832 - auc: 0.9864
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2779 - auc: 0.9875
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2671 - auc: 0.9891
Epoch 92/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2651 - auc: 0.9891
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2689 - auc: 0.9888
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2725 - auc: 0.9877
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2646 - auc: 0.9892
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2657 - auc: 0.9890
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2557 - auc: 0.9903
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [9]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 3.0469 - auc: 0.2341
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7890 - auc: 0.5226
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3834 - auc: 0.8195
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4681 - auc: 0.8516
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6155 - auc: 0.8591
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.6964 - auc: 0.8584
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7121 - auc: 0.8622
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6588 - auc: 0.8716
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 1.5432 - auc: 0.8808
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4340 - auc: 0.8814
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.2934 - auc: 0.9860
Epoch 89/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2842 - auc: 0.9879
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2839 - auc: 0.9876
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2863 - auc: 0.9867
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2855 - auc: 0.9869
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2933 - auc: 0.9854
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2850 - auc: 0.9864
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2800 - auc: 0.9879
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2820 - auc: 0.9869
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2831 - auc: 0.9869
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [10]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.6320 - auc: 0.7222
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5662 - auc: 0.7904
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3727 - auc: 0.8101
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2698 - auc: 0.8093
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1870 - auc: 0.8420
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0963 - auc: 0.8833
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0056 - auc: 0.9153
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9600 - auc: 0.9256
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8957 - auc: 0.9399
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.8277 - auc: 0.9544
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2269 - auc: 0.9921
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2309 - auc: 0.9913
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2222 - auc: 0.9915
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2307 - auc: 0.9904
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2280 - auc: 0.9914
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2317 - auc: 0.9911
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2262 - auc: 0.9911
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2225 - auc: 0.9924
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2202 - auc: 0.9923
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2270 - auc: 0.9917
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [11]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('7feature_remove_one.csv',index=False)
print(de)

               model       auc
0          all_model  0.949298
1  remove_AtomPairFP  0.931792
2      remove_Charge  0.950563
3       remove_Kappa  0.951808
4        remove_MAP4  0.945542
5         remove_MOE  0.951245
6    remove_MorganFP  0.949235
7    remove_Topology  0.951764
